In [182]:
import tfv.xarray
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [183]:
model_folder = Path(r'E:/WAMSI/')
model_file = 'csiem_B009_20221101_20240401_WQ.nc'

ds = xr.open_dataset(model_folder/model_file, decode_times=True, engine='netcdf4')
fv = ds.tfv

In [184]:
fv

<xarray.Dataset> Size: 90GB
Dimensions:      (Time: 6181, NumLayerFaces3D: 534016, NumCells2D: 30206,
                  NumCells3D: 503810)
Coordinates:
  * Time         (Time) datetime64[ns] 49kB 2022-11-02 ... 2024-03-31
Dimensions without coordinates: NumLayerFaces3D, NumCells2D, NumCells3D
Data variables:
    ResTime      (Time) datetime64[ns] 49kB 2022-11-02 ... 2024-03-31
    layerface_Z  (Time, NumLayerFaces3D) float32 13GB ...
    stat         (Time, NumCells2D) int32 747MB ...
    H            (Time, NumCells2D) float32 747MB ...
    D            (Time, NumCells2D) float32 747MB ...
    V_x          (Time, NumCells3D) float32 12GB ...
    V_y          (Time, NumCells3D) float32 12GB ...
    W            (Time, NumCells3D) float32 12GB ...
    SAL          (Time, NumCells3D) float32 12GB ...
    TEMP         (Time, NumCells3D) float32 12GB ...
    TRACE_1      (Time, NumCells3D) float32 12GB ...
Attributes:
    Origin:     Created by TUFLOWFV
    Type:       Cell-centred TUFLOWFV output
    spherical:  true
    Dry depth:  0.01

TUFLOW FV domain xarray accessor object

In [185]:
fv.W

<xarray.DataArray 'W' (Time: 6181, NumCells3D: 503810)> Size: 12GB
[3114049610 values with dtype=float32]
Coordinates:
  * Time     (Time) datetime64[ns] 49kB 2022-11-02 ... 2024-03-31
Dimensions without coordinates: NumCells3D
Attributes:
    long_name:  vertical velocity
    units:      m s^-1

In [4]:
%%time
## make regular grid at 0.001 resolution for Vx and Vy
grids = []
bbox = (115.63, -32.30, 115.80, -32.10) 
# Loop over each time step
for t in fv.Time.values:
    grid = fv.get_sheet_grid(
        time=str(t), 
        dx=0.001,        # horizontal resolution
        dy=0.001,        # vertical resolution
        datum='depth',
        bbox=bbox,
        limits=(0, 0.1)  # example limits, adjust as needed
    )
    
    # Select only Vx and Vy (if grid contains other variables)
    grid = grid[["TEMP", "V_x", "V_y"]]
    
    grids.append(grid)

# Combine along time
surface_grid = xr.concat(grids, dim="time")
surface_grid["time"] = fv.Time.values  # assign time coordinates
display(surface_grid)

<xarray.Dataset> Size: 5GB
Dimensions:    (time: 6181, latitude: 200, longitude: 171)
Coordinates:
  * time       (time) datetime64[ns] 49kB 2022-11-02 ... 2024-03-31
  * latitude   (latitude) float64 2kB -32.3 -32.3 -32.3 ... -32.1 -32.1 -32.1
  * longitude  (longitude) float64 1kB 115.6 115.6 115.6 ... 115.8 115.8 115.8
Data variables:
    TEMP       (time, latitude, longitude) float64 2GB nan 19.01 ... nan nan
    V_x        (time, latitude, longitude) float64 2GB nan -0.0308 ... nan nan
    V_y        (time, latitude, longitude) float64 2GB nan 0.3771 ... nan nan
Attributes:
    Origin:       2D Grid extracted from TUFLOW FV cell-centered output using...
    Type:         2D Grid from TUFLOW FV output
    Datum:        depth
    Limits:       (0, 0.1)
    Agg:          mean
    Grid method:  nearest

CPU times: total: 37min 15s
Wall time: 38min 43s


In [7]:
surface_grid.to_netcdf('surface_grid_temp_Vx_Vy.nc')

In [8]:
%%time
## make regular grid at 0.001 resolution for Vx and Vy
grids = []
bbox = (115.63, -32.30, 115.80, -32.10) 
# Loop over each time step
for t in fv.Time.values:
    grid = fv.get_sheet_grid(
        time=str(t), 
        dx=0.001,        # horizontal resolution
        dy=0.001,        # vertical resolution
        datum='height',
        bbox=bbox,
        limits=(0, 0.1)  # example limits, adjust as needed
    )
    
    # Select only Vx and Vy (if grid contains other variables)
    grid = grid[["TEMP", "V_x", "V_y"]]
    
    grids.append(grid)

# Combine along time
bottom_grid = xr.concat(grids, dim="time")
bottom_grid["time"] = fv.Time.values  # assign time coordinates
display(bottom_grid)

<xarray.Dataset> Size: 5GB
Dimensions:    (time: 6181, latitude: 200, longitude: 171)
Coordinates:
  * time       (time) datetime64[ns] 49kB 2022-11-02 ... 2024-03-31
  * latitude   (latitude) float64 2kB -32.3 -32.3 -32.3 ... -32.1 -32.1 -32.1
  * longitude  (longitude) float64 1kB 115.6 115.6 115.6 ... 115.8 115.8 115.8
Data variables:
    TEMP       (time, latitude, longitude) float64 2GB nan 19.01 ... nan nan
    V_x        (time, latitude, longitude) float64 2GB nan -0.0308 ... nan nan
    V_y        (time, latitude, longitude) float64 2GB nan 0.3771 ... nan nan
Attributes:
    Origin:       2D Grid extracted from TUFLOW FV cell-centered output using...
    Type:         2D Grid from TUFLOW FV output
    Datum:        height
    Limits:       (0, 0.1)
    Agg:          mean
    Grid method:  nearest

CPU times: total: 44min 57s
Wall time: 46min 42s


In [9]:
bottom_grid.to_netcdf('bottom_grid_temp_Vx_Vy.nc')